In [1]:
from ovcg import *
from fesol import *
import numpy as np
import scipy.io as sio
from interface import DynamicUnderRelaxation, InterfaceData, RelativeCovergenceMonitor

In [2]:
k=20
# problem discription:
# basically, with a fixed Pr=0.01 and Re=500, the k is the ratio between
# kappa_s/kappa_f, where kappa stands for conductivity coefficient.
assert k == 1 or k == 5 or k == 20

use_relax = True
use_solid_steady = True

# tolerance setting
tol = 1e-6
if tol > 1e-1:
    print('The tolerance is too large, this may lead to instability or low quality results')

In [3]:
# since this is a matching interface, we can create
# exact interface nodes mappings.
# this step is done offline
inodes = 100
s2f = np.zeros(inodes, dtype=int)
f2s = s2f.copy()
s2f[1] = inodes-1
start = inodes-2
for i in range(inodes-2):
    s2f[i+2] = start
    start = start - 1
count = 0
for i in range(inodes):
    f2s[s2f[i]] = i

In [4]:
# fluid solver
interface_grid = 3
interface_side = 0
interface_axis = 1

solverF = Cgins()
solverF.read_cmd('fluid_inputs/fluid_flat_plate%i.cmd' % k)
solverF.init_grid()
solverF.init_temperature_interface(
    grid=interface_grid,
    side=interface_side,
    axis=interface_axis
)
solverF.init_solver(debug=0, logname='fluid_outputs/dummy')
fluid_ofile = VtkFile('fluid_outputs/resFDSNavg%i.pvd' % k)

In [5]:
# solid solver
fixed_temp_tag = 2
adiabatic_tag = 3
interface_tag = 4

kappa = 100.0
rho = 1.0
Cp = 100.0
T_bot = 310.0

solverS = HeatSolver(kappa=kappa, rho=rho, Cp=Cp, steady=use_solid_steady, time_order=2)
solverS.output_file = 'solid_outputs/resFDSNavg%i.pvd' % k
solverS.load_mesh('solid_inputs/solid_plate.msh')
solverS.set_initial_condition(T_bot)
solverS.define_adiabatic_bd(tag=adiabatic_tag)
solverS.define_const_temperature_bd(tag=fixed_temp_tag, value=T_bot)
solverS.define_flux_interface(tag=interface_tag)
solverS.init_solver()

In [6]:
# coupling data and setups
fluxF = InterfaceData(size=inodes, value=0.0)
fluxS = InterfaceData(size=inodes, value=0.0)
tempF = InterfaceData(size=inodes, value=300.0)
tempS = InterfaceData(size=inodes, value=T_bot)
thetaT = InterfaceData(size=inodes, value=0.0)

under_relax = DynamicUnderRelaxation(init_omega=1.0)
conv_mntr = RelativeCovergenceMonitor(tol=tol)
conv_solution = RelativeCovergenceMonitor(tol=1e-5)

# maximum pc steps allowed
max_pc_steps = 200

In [7]:
# coupling timing information, use fixed time step for now
# Tfinal = 1.0
# N = 100
# dt = Tfinal/N
Nmax = 8000
t = 0.0
dt = 0.05
avg_pc_iterations = 0.0
flog = open('FDSNavg%i.log' % k, mode='w')
flog.write('Fluid Dirichlet with solid Neumann setting, study case: %i\n' % k)
flog.close()
flog = open('FDSNavg%i.log' % k, mode='a')

temps_prev = tempS.curr.copy()
fluxf_prev = fluxF.curr.copy()

In [8]:
# begin to couple
for step in range(Nmax):
    t += dt
    
    pc_counts = 0
    
    # back up solutions
    solverF.backup_current_solutions()
    solverS.backup()
    
    temps_prev[:] = tempS.curr
    fluxf_prev[:] = fluxF.curr
    
    while True:
        pc_counts = pc_counts + 1
        
        # back up previous interface value
        tempF.backup()
        fluxF.backup()
        tempS.backup()
        
        # advance fluid
        solverF.advance(t=t, dt=dt)
        # retrieve fluid interface flux
        fluxF.curr[:] = solverF.get_heat_flux_interface(
            grid=interface_grid,
            side=interface_side,
            axis=interface_axis
        )
        
        # mapping
        fluxS.curr[:] = 0.5*(fluxF.curr[f2s]+fluxf_prev[f2s])
        # update solid flux interface
        solverS.update_flux_interface(interface_tag, fluxS.curr)
        
        # advance solid
        solverS.advance(t=t, dt=dt)
        # retrieve solid interface temperature
        tempS.curr[:] = solverS.get_interface_temperature(tag=interface_tag)
        
        # mapping
        tempF.curr[:] = 0.5*(tempS.curr[s2f]+temps_prev[s2f])
        
        # update residual
        tempF.update_res()
        
        # test convergence
        is_conv = conv_mntr.determine_convergence(tempF)
        if is_conv or pc_counts >= max_pc_steps:
            solverF.put_temperature_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=tempF.curr
            )
            break
        else:
            # if not converge, then underrelaxation and update to fluid then restore
            if use_relax:
                under_relax.determine_omega(tempF)
                under_relax.update_solution(tempF)
            solverF.put_temperature_interface(
                grid=interface_grid,
                side=interface_side,
                axis=interface_axis,
                data=tempF.curr
            )
            solverF.restore_previous_solutions()
            solverS.restore()  

    # outputs
    msg = 'step=%i, pc_iterations=%i.' % (step, pc_counts)
    print(msg)
    flog.write(msg+'\n')
    avg_pc_iterations += pc_counts
    if step % 50 == 0 or step == Nmax-1:
        solverS.write()
        fluid_ofile.write(solverF, t)
    thetaT.backup()
    thetaT.curr[:] = (solverF.get_temperature_interface(3,0,1)-300)/10
    thetaT.update_res()
    is_conv = conv_solution.determine_convergence(thetaT)
    if is_conv:
        if step % 50 and step != Nmax-1:
            solverS.write()
            fluid_ofile.write(solverF, t)
        break
    
flog.close()

step=0, pc_iterations=5.
step=1, pc_iterations=5.
step=2, pc_iterations=4.
step=3, pc_iterations=4.
step=4, pc_iterations=4.
step=5, pc_iterations=4.
step=6, pc_iterations=4.
step=7, pc_iterations=4.
step=8, pc_iterations=4.
step=9, pc_iterations=4.
step=10, pc_iterations=4.
step=11, pc_iterations=4.
step=12, pc_iterations=3.
step=13, pc_iterations=3.
step=14, pc_iterations=3.
step=15, pc_iterations=3.
step=16, pc_iterations=3.
step=17, pc_iterations=3.
step=18, pc_iterations=3.
step=19, pc_iterations=3.
step=20, pc_iterations=3.
step=21, pc_iterations=3.
step=22, pc_iterations=3.
step=23, pc_iterations=3.
step=24, pc_iterations=3.
step=25, pc_iterations=3.
step=26, pc_iterations=3.
step=27, pc_iterations=3.
step=28, pc_iterations=3.
step=29, pc_iterations=3.
step=30, pc_iterations=3.
step=31, pc_iterations=3.
step=32, pc_iterations=3.
step=33, pc_iterations=3.
step=34, pc_iterations=3.
step=35, pc_iterations=3.
step=36, pc_iterations=3.
step=37, pc_iterations=3.
step=38, pc_iterations

In [9]:
theta=(solverF.get_temperature_interface(3,0,1)-300)/10
print(theta)
sio.savemat(
    'numeric_res/kFDSN_avg%i.mat' % k,
    {
        'theta': theta,
        'avg_pc_its': avg_pc_iterations/(step+1),
        'N': (step+1)
    }
)

[0.91511913 0.92186958 0.92672639 0.93069361 0.93406519 0.93692158
 0.93946547 0.94174065 0.94381898 0.94577335 0.9475626  0.94924693
 0.95081874 0.95230898 0.95370801 0.95503854 0.95629613 0.95749995
 0.95864421 0.95973219 0.96077322 0.9617659  0.96271957 0.96362958
 0.96449891 0.9653314  0.96613312 0.96690083 0.96763958 0.96834807
 0.96902702 0.9696803  0.97030835 0.97090998 0.97149249 0.97205039
 0.9725858  0.97310037 0.9735975  0.97407485 0.97453481 0.97497745
 0.97540232 0.97581099 0.97620401 0.97658204 0.97694506 0.97729466
 0.9776296  0.9779515  0.97825944 0.97855585 0.97883947 0.97911003
 0.97936963 0.9796164  0.97985268 0.98007667 0.98028939 0.98049146
 0.98068184 0.98086147 0.98102944 0.98118728 0.98133341 0.98146798
 0.98159236 0.98170415 0.98180518 0.98189416 0.98197108 0.98203614
 0.98208775 0.9821272  0.98215274 0.98216508 0.98216201 0.98214438
 0.98211169 0.98206142 0.98199495 0.981909   0.98180374 0.98167886
 0.9815308  0.98135895 0.98116014 0.98093398 0.98067552 0.9803